In [50]:
patch_train = "data/harem/coleccoes/CDPrimeiroHAREMprimeiroevento.xml"
#patch_train = "data/harem/coleccoes/test.xml"
patch_test = "data/harem/coleccoes/colSegundoHAREM.xml"
#patch_test = "data/harem/coleccoes/test_output.xml"

patch_out_test = "data/harem/out.xml"

In [2]:
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import xml.etree.ElementTree as et

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
def tokens(in_text):
    if in_text is None: return []
    return nltk.word_tokenize(in_text)

In [4]:
char_end_sentence = ['.', ';', '?', '!']

#Documents-> DOC-> EM, ALT->EM, OMITIDO->EM
tree = et.parse(patch_train)
doc_trees = tree.getroot()

sentences = []
for doc in doc_trees:
    
    sentence = []
    
    text = tokens(doc.text)
    entities = []
    
    for tag in doc:
        
        for t in text:
            if t in char_end_sentence:
                sentences.append(sentence)
                sentence = []
            else:
                sentence.append((t, 'O'))
        
        if tag.tag == 'EM':
            t_text = tokens(tag.text)
            first = True
            for t in t_text:
                if first:
                    first = False
                    sentence.append((t, tag.attrib['CATEG'].split('|')[0]+'-B'))
                else:    
                    sentence.append((t, tag.attrib['CATEG'].split('|')[0]+'-I'))
        
        elif tag.tag == 'ALT':
            t_text = tokens(tag.text)
            for a_tag in tag:
                end = False
                for t in t_text: 
                    if t == '|':
                        end = True
                        break
                    else:
                        sentence.append((t, 'O'))
                if end:
                    break
                first = True
                a_text = tokens(a_tag.text)
                for t in a_text:
                    if first:
                        first = False
                        sentence.append((t, a_tag.attrib['CATEG'].split('|')[0]+'-B'))
                    else:    
                        sentence.append((t, a_tag.attrib['CATEG'].split('|')[0]+'-I'))
                t_text = tokens(a_tag.tail)
        text = tokens(tag.tail)
    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []

In [5]:
size_s = len(sentences)
train_sents = sentences[:int(size_s-size_s/5)]
test_sents = sentences[int(size_s-size_s/5):]

In [6]:
print (size_s, len(train_sents), len(test_sents), len(train_sents) + len(test_sents))

4096 3276 820 4096


In [7]:
count = 0
for s in sentences:
    for ent in s:
        count += 1

In [ ]:
print (sentences[8])

In [8]:
def word2features(sent, i):
    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.replace(',', '').replace('.', '').isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [ ]:
sent2features(train_sents[0])

In [9]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

Wall time: 197 ms


In [10]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 15.8 s


In [11]:
labels = list(crf.classes_)
#labels.remove('O')
labels

['ORGANIZACAO-B',
 'O',
 'ORGANIZACAO-I',
 'ABSTRACCAO-B',
 'TEMPO-B',
 'TEMPO-I',
 'LOCAL-B',
 'VALOR-B',
 'VALOR-I',
 'PESSOA-B',
 'PESSOA-I',
 'LOCAL-I',
 'ABSTRACCAO-I',
 'ACONTECIMENTO-B',
 'ACONTECIMENTO-I',
 'OBRA-B',
 'OUTRO-B',
 'OUTRO-I',
 'OBRA-I',
 'COISA-B',
 'COISA-I',
 'OBJECTO-B']

In [12]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.94532772215966565

In [13]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                 precision    recall  f1-score   support

              O      0.985     0.992     0.988     16079
        VALOR-B      0.776     0.725     0.750        91
        VALOR-I      0.694     0.883     0.777        77
      OBJECTO-B      0.000     0.000     0.000         1
         OBRA-B      0.348     0.235     0.281        34
         OBRA-I      0.250     0.292     0.269        72
   ABSTRACCAO-B      0.489     0.262     0.341        84
   ABSTRACCAO-I      0.593     0.308     0.405       104
ACONTECIMENTO-B      0.200     0.118     0.148        17
ACONTECIMENTO-I      0.323     0.189     0.238        53
        TEMPO-B      0.810     0.780     0.795        82
        TEMPO-I      0.936     0.846     0.889        52
       PESSOA-B      0.550     0.518     0.533       139
       PESSOA-I      0.566     0.710     0.630       138
        LOCAL-B      0.693     0.574     0.628       263
        LOCAL-I      0.520     0.503     0.511       177
        COISA-B      0.200    

In [14]:
char_no_space_before = ['.', ';', ':', '?', '!', ',', u'«', u'»']

def add_words_out(word, sentenc):
    if sentenc is None or len(sentenc) == 0:
        sentenc = word
    else:
        if word in char_no_space_before:
            sentenc = sentenc + word
        else:
            sentenc = sentenc + ' ' + word
    return sentenc

def join_words_out(words):
    ret = ''
    for w in words:
        if len(ret) == 0:
            ret = w
        else:
            if w in char_no_space_before:
                ret = ret+w
            else:
                ret = ret+' '+w
    return ret

In [15]:
def predict(sent, model):
    ret = sent2features([[x] for x in sent])
    #for i in ret:
    #    print(i['word.lower()'],)
    return model.predict([ret])[0]

In [16]:
a = "O IRA esteve esta semana na ofensiva, paralisando o aeroporto de Londres e causando prejuízos à temporada turística britânica"
predict(tokens(a), crf)

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOCAL-B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [51]:
%%time
test_tree = et.parse(patch_test)
test_docs = test_tree.getroot()

out_tree = et.Element("colHAREM")
out_tree.set('versao','CAGE2-SegundoHAREM-4')
ids = 1

# Pra cada Documento
for test_doc in test_docs:
    
    out_doc = et.SubElement(out_tree, "DOC")
    out_doc.set('DOCID', test_doc.attrib['DOCID'])
    
    # Pra cada Parágrafo
    for test_p in test_doc:
        
        out_p = et.SubElement(out_doc, 'P')
        
        test_text = tokens(test_p.text)
        
        # Pra cada Token
        sentenca = []
        out_em = None  # Onde deve ser escrito as não Entidades
        for test_t in test_text:
            
            if test_t in char_end_sentence:
                # Fim de sentença - Predict
                ret = predict(sentenca, crf)
                
                #Pra cada posição da sentenca
                out_s_text = ''
                for i in range(len(sentenca)):
                    s_r = ret[i]
                    s_t = sentenca[i]
                    
                    if s_r == 'O':
                        # Não é Entidade
                        out_s_text = add_words_out(s_t, out_s_text)
                    else:
                        # É Entidade
                        if out_em is None:
                            out_p.text = add_words_out(out_s_text, out_p.text)
                        else:
                            out_em.tail = add_words_out(out_s_text, out_em.tail)
                        out_s_text = ''
                        
                        if '-B' in s_r:
                            if out_em is None:
                                out_p.text = out_p.text
                            else:
                                out_em.tail = out_em.tail
                            out_em = et.SubElement(out_p, 'EM')
                            out_em.set('ID', str(ids))
                            out_em.set('CATEG', s_r[:s_r.index('-')])
                            ids += 1
                        out_em.text = add_words_out(s_t, out_em.text)
                            
                if len(out_s_text) > 0:
                    #Texto pós entidades (ou sem entidade) da sentenca
                    if out_em is None:
                        out_p.text = add_words_out(out_s_text, out_p.text)
                    else:
                        out_em.tail = add_words_out(out_s_text, out_em.tail)
                # Adicionar end charactere
                if out_em is None:
                    out_p.text = add_words_out(test_t, out_p.text)
                else:
                    out_em.tail = add_words_out(test_t, out_em.tail)
                sentenca = []
                
            else:
                # Sentença não acabou
                sentenca.append(test_t)
            
        # Sobrou sentenca (ex: parágrafo sem ponto final)
        if len(sentenca) > 0:
            if out_em is None:
                out_p.text = add_words_out(join_words_out(sentenca), out_p.text)
            else:
                out_em.tail = add_words_out(join_words_out(sentenca), out_em.tail)

tree = et.ElementTree(out_tree)
tree.write(patch_out_test, encoding="UTF-8")

Wall time: 13.1 s
